Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

In [3]:
file = open('checkins.dat', 'r')
for i in range(10):
    f = file.readline()
    print(f)

file.close()

   id    | user_id | venue_id |     latitude      |     longitude     |     created_at      

---------+---------+----------+-------------------+-------------------+---------------------

  984301 | 2041916 |     5222 |                   |                   | 2012-04-21 17:39:01

  984222 |   15824 |     5222 |        38.8951118 |       -77.0363658 | 2012-04-21 17:43:47

  984315 | 1764391 |     5222 |                   |                   | 2012-04-21 17:37:18

  984234 |   44652 |     5222 |         33.800745 |         -84.41052 | 2012-04-21 17:43:43

  984249 | 2146840 |     5222 |                   |                   | 2012-04-21 17:42:58

  984268 | 2146843 |     5222 |                   |                   | 2012-04-21 17:42:38

  984281 | 2146846 |     5222 |                   |                   | 2012-04-21 17:39:40

  984291 |  105054 |     5222 |        45.5234515 |      -122.6762071 | 2012-04-21 17:39:22



Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [102]:
import pandas as pd
import numpy as np

In [65]:
data = pd.read_csv('checkins.dat', sep='|')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [66]:
data.columns
#все наименования содержат кучу пробелов, надо их удалить

Index(['   id    ', ' user_id ', ' venue_id ', '     latitude      ',
       '     longitude     ', '     created_at      '],
      dtype='object')

In [71]:
columns_list = list(data.columns)
for i in range(len(columns_list)):
    columns_list[i] = columns_list[i].strip()

data.columns = columns_list
data = data.dropna() #сразу удалим первые строки не являющиеся данными с Null значениями

In [72]:
data['longitude'] = data['longitude'].apply(lambda x: x.strip()) #обрезаем пробелы в данных долготы
data['latitude'] = data['latitude'].apply(lambda x: x.strip()) #обрезаем пробелы в данных широты
data = data.drop(data[data['latitude'] == ''].index) #удаляем все строки где отсутствуют данные о локации

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [73]:
#Получили такую таблицу
data

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43
8,984291,105054.0,5222.0,45.5234515,-122.6762071,2012-04-21 17:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-21 17:35:46
11,984232,93870.0,380645.0,33.4483771,-112.0740373,2012-04-21 17:38:18
...,...,...,...,...,...,...
1021960,955561,626076.0,20073.0,40.8501002,-73.8662464,2012-04-13 09:56:48
1021961,955892,674797.0,2297.0,33.7489954,-84.3879824,2012-04-13 10:56:03
1021962,956377,845102.0,11195.0,42.7653662,-71.467566,2012-04-13 12:08:45
1021963,956119,1139114.0,29488.0,42.439479,-83.7438303,2012-04-13 11:36:44


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

#### Примечание:
на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

In [83]:
test_data = data.head(100000) #берем первые 100000 строк для кластеризации
test_data.shape

(100000, 6)

In [85]:
from sklearn.cluster import MeanShift
ms = MeanShift(bandwidth=0.1)

In [86]:
ms.fit(test_data.loc[:,'latitude':'longitude']) #обучаем на данных о геолокации

MeanShift(bandwidth=0.1)

In [110]:
labels = ms.labels_ #все наши метки
cluster_centers = ms.cluster_centers_ # центры полученных кластеров
n_clusters_ = len(cluster_centers) # количество полученных кластеров

val, counts = np.unique(labels, return_counts=True)
print(val, counts)

[   0    1    2 ... 3228 3229 3230] [12506  4692  3994 ...     1     1     1]


In [115]:
len(cluster_centers)

3231

In [130]:
new_cluster_centers = []
j = 0
for i in range(len(counts)):
    if counts[i] > 15:
        new_cluster_centers.append(cluster_centers[i]) # в новый список центров добавляем только те, где больше 15 элементов
        j += 1

In [129]:
len(new_cluster_centers) #количество новых центров уменьшилось

592

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

In [132]:
office_list = [[33.751277, -118.188740], #(Los Angeles)
              [25.867736, -80.324116], # (Miami)
              [51.503016, -0.075479], # (London)
              [52.378894, 4.885084], # (Amsterdam)
              [39.366487, 117.036146], # (Beijing)
              [-33.868457, 151.205134]] # (Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [133]:
# формула расчета расстояния будет обычной евклидовой метрикой
def get_dist(office_co, ms_co):
    return np.sqrt((office_co[0] - ms_co[0]) ** 2 + (office_co[1] - ms_co[1]) ** 2)

In [137]:
best_dist = pd.DataFrame(columns = ['min_dist', 'latitude', 'longitude'])
for i in range(len(new_cluster_centers)):
    min_dist = []
    for j in range(len(office_list)):
        min_dist.append(get_dist(new_cluster_centers[i], office_list[j]))
        
    best_dist.loc[i] = [min(min_dist), new_cluster_centers[i][0], new_cluster_centers[i][1]]

In [138]:
best_dist

,min_dist,latitude,longitude
0,16.143720,40.717716,-73.991835
1,6.193959,33.449438,-112.002140
2,6.294241,33.446380,-111.901888
3,17.598580,41.878244,-87.629843
4,5.772048,37.688682,-122.409330
...,...,...,...
587,17.161408,41.577224,-73.415723
588,16.733992,41.220398,-73.666619
589,17.721330,41.618532,-88.445568
590,13.745649,39.249469,-77.182127


In [140]:
best_dist.sort_values('min_dist').head(20)

,min_dist,latitude,longitude
417,0.007835,-33.860630,151.204776
369,0.009353,52.372964,4.892317
416,0.022674,25.845672,-80.318891
58,0.050058,51.502991,-0.125537
51,0.070848,33.809878,-118.148924
29,0.134109,25.785812,-80.217938
167,0.167406,25.705350,-80.283429
92,0.188876,26.010098,-80.199991
87,0.195779,33.888325,-118.048928
42,0.211811,33.872986,-118.362091


Ответом получаются координаты -33.868457, 151.205134 и это, если сравнить с табличкой данных офисов - город Сидней